In [28]:
import collections
import itertools
import pandas as pd
import scipy.stats as st
import statsmodels.tools.numdiff as smnd
import numpy as np
import numba
import random
import math
from copy import copy, deepcopy
import corner

random.seed()

def mh_step(x, log_post, log_post_current, sigma, args=()):
    """
    Parameters
    ----------
    x : ndarray, shape (n_variables,)
        The present location of the walker in parameter space.
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    log_post_current : float
        The current value of the log posterior.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.

    Returns
    -------
    x_out : ndarray, shape (n_variables,)
        The position of the walker after the Metropolis-Hastings
        step. If no step is taken, returns the inputted `x`.
    log_post_updated : float
        The log posterior after the step.
    accepted : bool
        True is the proposal step was taken, False otherwise.
    """
    nextPoint = []
    s = np.random.normal(x[0], sigma)
    s1 = np.random.normal(x[1], sigma)
    #print (s,s1)
    firstPoint = s
    secondPoint = s1
    nextPoint.append(firstPoint)
    nextPoint.append(secondPoint)

    logmetropolisRatio = log_post(nextPoint, *args) / log_post_current
    #print (log_post_current)
    #print (metropolisRatio)
    n = random.uniform(0,1)
    if (logmetropolisRatio >= 0 or np.exp(logmetropolisRatio) <= n):
        return nextPoint, log_post(np.array(nextPoint), *args), True
    else:
        return x, log_post_current, False
    
def mh_sample(log_post, x0, sigma, args=(), n_burn=100000, n_steps=500000,
              variable_names=None):
    """
    Parameters
    ----------
    log_post : function
        The function to compute the log posterior. It has call
        signature `log_post(x, *args)`.
    x0 : ndarray, shape (n_variables,)
        The starting location of a walker in parameter space.
    sigma : ndarray, shape (n_variables, )
        The standard deviations for the proposal distribution.
    args : tuple
        Additional arguments passed to `log_post()` function.
    n_burn : int, default 1000
        Number of burn-in steps.
    n_steps : int, default 1000
        Number of steps to take after burn-in.
    variable_names : list, length n_variables
        List of names of variables. If None, then variable names
        are sequential integers.
    
    Returns
    -------
    output : DataFrame
        The first `n_variables` columns contain the samples.
        Additionally, column 'lnprob' has the log posterior value
        at each sample.
    """
    samples = []
    lnprob = []
    finalPoint = x0
    new_mu, new_cov = args
    logPostCurrent = log_post(x0, *args)
    isAccepted = True
    for i in range(n_burn):
        finalPoint, logPostCurrent, isAccepted = mh_step(np.array(finalPoint), log_post, logPostCurrent, sigma, args)
        
    for j in range(n_steps):
        finalPoint, logPostCurrent, isAccepted = mh_step(np.array(finalPoint), log_post, logPostCurrent, sigma, args)
        samples.append(finalPoint)
        lnprob.append(logPostCurrent)
        #print (finalPoint)
    d = {'Samples': samples, 'Log Posterior Value': lnprob}
    df = pd.DataFrame(data=d)
    return df
        
mu = np.array([10.0, 20.0])
cov = np.array([[4, -2],[-2, 6]])
inv_cov = np.linalg.inv(cov)

#@numba.jit(nopython=True)
def log_test_distribution(x, mu, inv_cov):
    """
    Unnormalized log posterior of a multivariate Gaussian.
    """
    return -np.dot((x-mu), np.dot(inv_cov, (x-mu))) / 2

x0 = np.array([5.0, 15.0])
_out = mh_sample(log_test_distribution,x0 , 1.0, args=(mu, inv_cov))
#print (_out['Samples'])
totalFirst = 0
totalSecond = 0
count = 0
for i in _out['Samples']:
    totalFirst += i[0]
    totalSecond += i[1]
    count += 1
#print (totalFirst/count)
#print (totalSecond / count)



In [13]:
_out['Samples'][0]

-49.36897800143166

In [19]:
samples1 = []
samples2 = []
for i in range(len(_out['Samples'])):
    samples1.append(_out['Samples'][i][0])
    samples2.append(_out['Samples'][i][1])
_out['Samples_1'] = pd.Series(samples1)
_out['Samples_2'] = pd.Series(samples2)
del _out['Samples']

In [21]:
_out.head()

,Log Posterior Value,Samples_1,Samples_2
0,-484.000519,-49.368978,28.847965
1,-490.841173,-49.534663,27.905454
2,-505.722525,-50.149077,26.948019
3,-535.403260,-51.282403,24.958202
4,-537.657262,-51.245740,24.397468


In [24]:
i = 0
first = []
second = []
for i in range(len(_out['Samples_1'])):
    first.append(_out['Samples_1'][i])
    second.append(_out['Samples_2'][i])


In [25]:
import matplotlib.pyplot as plt
#plt.plot(first,second)

In [27]:
cols = ['Log Posterior Value', 'Samples_1', 'Samples_2']
corner.corner(_out, labels=cols, bins=100);
plt.show()

In [ ]:
d